# Topic Modeling
### Without Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 17:59:26.215100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 17:59:26.215146: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [20]:
n_components = 25 #Topics To generate
n = 40 #Transcripts to use
max_df=20
min_df=3
n_terms = 10

## Load Data

In [3]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [4]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                text = ' '.join(y['UTTERANCES'])

                text = clean_text(text)
                list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [5]:
df = create_df_multi(n)
print(df.shape)
df.head()

(105048, 1)


,0
0,welcome evening to show bunch of ass voice act...
1,welcome back so first foremost get couple anno...
2,so
3,slightly
4,go worry about want apologize in advance to fo...


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [6]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray().transpose(),
                   index=tfidf.get_feature_names_out())
tfidf_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray().transpose(),
                   index=cv.get_feature_names_out())
cv_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [21]:
def print_top_terms(n_components, topics, terms, n_terms = 10):
    for x in range(n_components):
        topic = x
        components = topics[:,topic]
        top_term_indices = components.argsort()[-n_terms:]
        top_terms = np.array(terms)[top_term_indices]
        
        print(f'Topic {x}:\t{top_terms.tolist()}')


## PCA

In [28]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 1:	['combine', 'nostril', 'necessity', 'ware', 'sage', 'breeze', 'drape', 'ceremony', 'port', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['flying', 'hike', 'manipulation', 'tradition', 'knack', 'meditate', 'childhood', 'truly', 'impression', 'headmaster']
Topic 4:	['potential', 'devise', 'abdoman', 'orcs', 'puzzle', 'reign', 'demand', 'devour', 'impression', 'headmaster']
Topic 5:	['kneel', 'brighten', 'intensity', 'atop', 'chip', 'stain', 'account', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['commotion', 'snuck', 'cobra', 'midst', 'boon', 'purely', 'follower', 'masquerade', 'ti', 'yuan']
Topic 8:	['unfold', 'user', 'orig

In [29]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['keg', 'mimic', 'text', 'grieve', 'socket', 'lycan', 'invisibility', 'toast', 'tary', 'eyeball']
Topic 1:	['sizzle', 'cake', 'y', 'disarm', 'marching', 'text', 'toast', 'vomit', 'gingerly', 'grieve']
Topic 2:	['tick', 'barrack', 'healer', 'queen', 'yesterday', 'grieve', 'tary', 'gingerly', 'locket', 'theater']
Topic 3:	['y', 'grieve', 'grace', 'keg', 'lately', 'mimic', 'text', 'gingerly', 'lycan', 'locket']
Topic 4:	['advisor', 'pen', 'tearfully', 'toast', 'theater', 'grieve', 'yesterday', 'invisibility', 'lately', 'lycan']
Topic 5:	['sizzle', 'tablet', 'okay', 'tary', 'gone', 'advisor', 'text', 'grieve', 'locket', 'invisibility']
Topic 6:	['disperse', 'alert', 'holding', 'queen', 'lycan', 'pen', 'lately', 'sure', 'invisibility', 'gingerly']
Topic 7:	['invisibility', 'grieve', 'mimic', 'bless', 'gone', 'locket', 'pen', 'toast', 'tary', 'lately']
Topic 8:	['bless', 'sure', 'grieve', 'text', 'mimic', 'invisibility', 'locket', 'tearfully', 'okay', 'pen']
Topic 9:	['advisor', 'in

## LDA

In [26]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['chatroom', 'beetle', 'hex', 'holding', 'toast', 'instantly', 'gesture', 'y', 'tablet', 'chant']
Topic 1:	['r', 'lately', 'lycan', 'nudge', 'headmaster', 'depth', 'eyeball', 'speech', 'stronghold', 'grey']
Topic 2:	['basis', 'gleam', 'rpg', 'spinning', 'flavor', 'barrack', 'investigate', 'text', 'component', 'guidance']
Topic 3:	['puzzle', 'keg', 'alert', 'electricity', 'bread', 'crouch', 'longsword', 'gash', 'loosely', 'vomit']
Topic 4:	['detect', 'clientele', 'tighten', 'rapidly', 'missile', 'moisture', 'queen', 'warn', 'lollipop', 'healer']
Topic 5:	['rally', 'estate', 'sailor', 'relic', 'maul', 'linger', 'sway', 'scare', 'obscure', 'marry']
Topic 6:	['caduceus', 'destination', 'lumber', 'sneeze', 'genuinely', 'drown', 'tary', 'bark', 'lemure', 'pew']
Topic 7:	['revenge', 'temperature', 'char', 'clay', 'goes', 'n', 'misfire', 'carriage', 'bath', 'mini']
Topic 8:	['hut', 'rebellion', 'stat', 'dawn', 'drain', 'loop', 'gingerly', 'alcohol', 'sundown', 'develop']
Topic 9:	['pr

In [27]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['vary', 'tether', 'pew', 'rag', 'necromancy', 'carving', 'lollipop', 'marching', 'yesterday', 'pen']
Topic 1:	['glad', 'grappling', 'kima', 'occasion', 'download', 'linger', 'efreet', 'maul', 'bless', 'tearfully']
Topic 2:	['backhand', 'tax', 'dancing', 'situate', 'wrong', 'signing', 'barrack', 'tablet', 'warn', 'disperse']
Topic 3:	['surrounding', 'rapidly', 'judgment', 'artist', 'relief', 'stink', 'alcohol', 'keg', 'sure', 'tary']
Topic 4:	['gas', 'gaming', 'obscure', 'latch', 'attune', 'vibrate', 'corrupt', 'mini', 'goes', 'mimic']
Topic 5:	['gnoll', 'stitch', 'electricity', 'weakness', 'stronghold', 'loosely', 'sundown', 'missile', 'opposite', 'percent']
Topic 6:	['lumber', 'schedule', 'octopus', 'fortress', 'stomp', 'exhaustion', 'cake', 'value', 'intentionally', 'toast']
Topic 7:	['male', 'compare', 'clicking', 'custom', 'fool', 'casting', 'occur', 'concern', 'pole', 'gone']
Topic 8:	['carriage', 'jug', 'digit', 'midnight', 'fortune', 'customer', 'instantly', 'duration'

## SVD

In [24]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['contemplate', 'ware', 'cling', 'sage', 'breeze', 'port', 'creator', 'drape', 'ceremony', 'looter']
Topic 1:	['practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['flying', 'hike', 'manipulation', 'tradition', 'knack', 'meditate', 'childhood', 'truly', 'impression', 'headmaster']
Topic 4:	['divergence', 'den', 'potential', 'abdoman', 'devise', 'orcs', 'puzzle', 'reign', 'demand', 'devour']
Topic 5:	['apex', 'brighten', 'intensity', 'chip', 'atop', 'stain', 'account', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['snuck', 'cobra', 'midst', 'boon', 'purely', 'follower', 'carriage', 'masquerade', 'ti', 'yuan']
Topic 8:	['unfold', 'user', 'origin', 'sub

In [25]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['component', 'tablet', 'holding', 'warn', 'keg', 'tary', 'gone', 'gingerly', 'invisibility', 'eyeball']
Topic 1:	['nudge', 'deadeye', 'sadly', 'invisibility', 'mimic', 'text', 'theater', 'queen', 'tary', 'grieve']
Topic 2:	['holding', 'gone', 'warn', 'dmed', 'rpg', 'grieve', 'invisibility', 'theater', 'locket', 'lycan']
Topic 3:	['disarm', 'drown', 'sure', 'badass', 'y', 'invisibility', 'vomit', 'tearfully', 'gingerly', 'theater']
Topic 4:	['instantly', 'warhammer', 'disarm', 'mimic', 'pen', 'sure', 'yesterday', 'gingerly', 'grieve', 'locket']
Topic 5:	['locket', 'gone', 'disperse', 'y', 'disarm', 'lately', 'vomit', 'mimic', 'gingerly', 'invisibility']
Topic 6:	['marching', 'warn', 'queen', 'text', 'opposite', 'gone', 'toast', 'lately', 'gingerly', 'lycan']
Topic 7:	['disperse', 'advisor', 'tablet', 'y', 'lately', 'marching', 'locket', 'toast', 'vomit', 'tary']
Topic 8:	['rpg', 'advisor', 'casino', 'sure', 'marching', 'sailor', 'pen', 'okay', 'vomit', 'text']
Topic 9:	['yeste

## NMF

In [22]:
nmf = NMF(n_components=n_components, max_iter=500)
topics = nmf.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Topic 0:	['barker', 'storefront', 'combine', 'nostril', 'necessity', 'drape', 'breeze', 'ware', 'sage', 'port']
Topic 1:	['creator', 'practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'ceremony', 'cling', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['childhood', 'expectation', 'tradition', 'flying', 'hike', 'manipulation', 'meditate', 'truly', 'impression', 'headmaster']
Topic 4:	['nameless', 'divergence', 'potential', 'den', 'abdoman', 'orcs', 'puzzle', 'reign', 'demand', 'devour']
Topic 5:	['recovery', 'guardian', 'kneel', 'intensity', 'brighten', 'chip', 'stain', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['commotion', 'snuck', 'creator', 'cobra', 'midst', 'boon', 'purely', 'follower', 'ti', 'yuan']
Topic 8:	['unfold', 'subscri

In [23]:
nmf = NMF(n_components=n_components, max_iter=400)
topics = nmf.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


KeyboardInterrupt: 